## Ex1 Use data from Danmarks Statistik - Databanken
1. Go to https://www.dst.dk/da/Statistik/brug-statistikken/muligheder-i-statistikbanken/api#testkonsol
2. Open 'Konsol' and click 'Start Konsol'
3. In the console at pt 1: choose 'Retrieve tables', pt 2: choose get request and json format and pt 3: execute:
  1. check the result
  2. in the code below this same get request is used to get information about all available data tables in 'databanken'. 
4. Change pt. 1 in the console to 'Retrieve data', pt 2: 'get request' and Table id: 'FOLK1A', format: csv, delimiter: semicolon and click: 'Variable and value codes' and choose some sub categories (Hint: hover over the codes to see their meaning). Finally execute and see what data you get.
5. With data aggregation and data visualization answer the following questions:
  1. What is the change in pct of divorced danes from 2008 to 2020?
  2. Which of the 5 biggest cities has the highest percentage of 'Never Married' in 2020?
  3. Show a bar chart of changes in marrital status in Copenhagen from 2008 till now
  4. Show 2 plots in same figure: 'Married' and 'Never Married' for all ages in DK in 2020 (Hint: x axis is age from 0-125, y axis is how many people in the 2 categories). Add lengend to show names on graphs

## 1 - 4)

In [6]:
import pandas as pd
df = pd.read_csv('https://api.statbank.dk/v1/data/FOLK1A/CSV?lang=en&delimiter=Semicolon&OMR%C3%85DE=*&K%C3%98N=1%2C2&CIVILSTAND=U%2CG%2CE%2CF&Tid=*', sep=';')
df

,OMRÅDE,KØN,CIVILSTAND,TID,INDHOLD
0,All Denmark,Men,Never married,2008Q1,1366147
1,All Denmark,Men,Never married,2008Q2,1371341
2,All Denmark,Men,Never married,2008Q3,1372281
3,All Denmark,Men,Never married,2008Q4,1374057
4,All Denmark,Men,Never married,2009Q1,1377636
...,...,...,...,...,...
49555,Aalborg,Women,Divorced,2021Q3,10437
49556,Aalborg,Women,Divorced,2021Q4,10446
49557,Aalborg,Women,Divorced,2022Q1,10506
49558,Aalborg,Women,Divorced,2022Q2,10552


## 5) With data aggregation and data visualization answer the following questions:

### 1. What is the change in pct of divorced danes from 2008 to 2020?

In [18]:
df.iloc[5:100,2]

5         Never married
6         Never married
7         Never married
8         Never married
9         Never married
            ...        
95    Married/separated
96    Married/separated
97    Married/separated
98    Married/separated
99    Married/separated
Name: CIVILSTAND, Length: 95, dtype: object

### 2. Which of the 5 biggest cities has the highest percentage of 'Never Married' in 2020?

### 3. Show a bar chart of changes in marrital status in Copenhagen from 2008 till now

###  4. Show 2 plots in same figure: 'Married' and 'Never Married' for all ages in DK in 2020 (Hint: x axis is age from 0-125, y axis is how many people in the 2 categories). Add lengend to show names on graphs

## Ex2 Use another table (extra)
Choose any of the other tables in 'databanken' to find interesting data.
1. Collect the data
2. Pose 5 or more interesing questions to the data
3. Answer the questions by aggregating the data
4. Illustrate the answers with visual plots

In [1]:
import pandas as pd 
url = 'https://api.statbank.dk/v1/tables'
dst = pd.read_json(url)
dst.to_csv('dk-stat-all-tables.csv', encoding='utf-8', index=False)
dst[:20]

,id,text,unit,updated,firstPeriod,latestPeriod,active,variables
0,FOLK1A,Folketal den 1. i kvartalet,Antal,2022-08-11T08:00:00,2008K1,2022K3,True,"[område, køn, alder, civilstand, tid]"
1,FOLK1AM,Folketal den 1. i måneden,Antal,2022-09-07T08:00:00,2021M10,2022M08,True,"[område, køn, alder, tid]"
2,FOLK3,Folketal 1. januar,Antal,2022-02-11T08:00:00,2008,2022,True,"[fødselsdag, fødselsmåned, fødselsår, tid]"
3,FOLK3FOD,Folketal 1. januar,Antal,2022-03-18T08:00:00,2008,2022,True,"[fødselsdag, fødselsmåned, fødeland, tid]"
4,BEF5,Folketal pr. 1. januar,Antal,2022-02-11T08:00:00,1990,2022,True,"[køn, alder, fødeland, tid]"
5,FT,Folketal (summariske tal fra folketællinger),Antal,2022-02-11T08:00:00,1769,2022,True,"[hovedlandsdele, tid]"
6,BY1,Folketal 1. januar,Antal,2022-05-27T08:00:00,2010,2022,True,"[byområder og landdistrikter, alder, køn, tid]"
7,BY2,Folketal 1. januar,Antal,2022-05-27T08:00:00,2010,2022,True,"[kommune, bystørrelse, alder, køn, tid]"
8,BY3,Folketal 1. januar,-,2022-07-14T08:00:00,2017,2022,True,"[byområder og landdistrikter, folketal, areal ..."
9,KM1,Folketal den 1. i kvartalet,Antal,2022-08-11T08:00:00,2007K1,2022K3,True,"[sogn, folkekirkemedlemsskab, tid]"


In [2]:
pd.read_csv('https://api.statbank.dk/v1/data/FOLK1A/CSV?delimiter=Semicolon&OMR%C3%85DE=*&K%C3%98N=1%2C2&ALDER=*&CIVILSTAND=U%2CG&Tid=2020K4',sep=';')

,OMRÅDE,KØN,ALDER,CIVILSTAND,TID,INDHOLD
0,Hele landet,Mænd,I alt,Ugift,2020K4,1520367
1,Hele landet,Mænd,I alt,Gift/separeret,2020K4,1072984
2,Hele landet,Mænd,0 år,Ugift,2020K4,31397
3,Hele landet,Mænd,0 år,Gift/separeret,2020K4,0
4,Hele landet,Mænd,1 år,Ugift,2020K4,31786
...,...,...,...,...,...,...
53335,Aalborg,Kvinder,123 år,Gift/separeret,2020K4,0
53336,Aalborg,Kvinder,124 år,Ugift,2020K4,0
53337,Aalborg,Kvinder,124 år,Gift/separeret,2020K4,0
53338,Aalborg,Kvinder,125 år,Ugift,2020K4,0
